In [2]:
import torch
import torch.nn as nn

In [91]:
class ResidualModule(nn.Module): # for resnet 18 and resnet 34
    def __init__(self, input_channel, output_channel, stride=1):
        super().__init__()
        self.module1 = nn.Conv2d(input_channel, output_channel, 3, stride, padding=1)
        self.module2 = nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.layers = nn.Sequential(
            self.module1,
            nn.BatchNorm2d(output_channel),
            nn.ReLU(inplace=True),
            self.module2,
            nn.BatchNorm2d(output_channel),
        )

        if (input_channel != output_channel):
            self.shortcut = nn.Sequential(
                nn.Conv2d(input_channel, output_channel, 1, stride),
                nn.BatchNorm2d(output_channel),
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, X):
        return nn.ReLU(inplace=True)(self.shortcut(X) + self.layers(X))

In [79]:
class ResidualModuleBottleNeck(nn.Module): # for resnet >=50
    def __init__(self, input_channel, output_channel, stride=1):
        super().__init__()
        self.module1 = nn.Conv2d(input_channel, output_channel / 4, 1)
        self.module2 = nn.Conv2d(output_channel / 4, output_channel / 4, 3, stride, padding=1)
        self.module3 = nn.Conv2d(output_channel / 4, output_channel, 1)
        self.layers = nn.Sequential(
            self.module1,
            nn.BatchNorm2d(output_channel / 4),
            nn.ReLU(),
            self.module2,
            nn.BatchNorm2d(output_channel / 4),
            nn.ReLU(),
            self.module3,
            nn.BatchNorm2d(output_channel),
        )

        if (input_channel != output_channel):
            self.shortcut = nn.Sequential(
                nn.Conv2d(input_channel, output_channel, 1, stride),
                nn.BatchNorm2d(output_channel),
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, X):
        return nn.ReLU(self.shortcut(X) + self.layers(X))

In [80]:
class ConvLayer(nn.Module):
    def __init__(self, residual_module_type, input_channel, output_channel, depth, first_stride=1):
        super().__init__()
        self.modules = [residual_module_type(input_channel, output_channel, first_stride)]
        for i in range(1, depth):
            self.modules.append(residual_module_type(output_channel, output_channel))
        self.layers = nn.ModuleList(self.modules)

    def forward(self, X):
        for module in self.layers:
            X = module(X)
        return X

In [104]:
class ResNet(nn.Module):
    def __init__(self, residual_module_type, residual_module_depth, num_classes=2):
        super().__init__()

        self.init_block = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2, padding=3),
            nn.MaxPool2d(3, 2)
        )

        self.layer1 = ConvLayer(residual_module_type, 64, 64, residual_module_depth[0])
        self.layer2 = ConvLayer(residual_module_type, 64, 128, residual_module_depth[1], first_stride=2)
        self.layer3 = ConvLayer(residual_module_type, 128, 256, residual_module_depth[2], first_stride=2)
        self.layer4 = ConvLayer(residual_module_type, 256, 512, residual_module_depth[3], first_stride=2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Linear(512, num_classes),
            nn.Softmax(dim=1),
        )

    def forward(self, X):
        X = self.init_block(X)
        X = self.layer1(X)
        X = self.layer2(X)
        X = self.layer3(X)
        X = self.layer4(X)
        X = self.avg_pool(X)
        X = X.view(X.size(0), -1)
        X = self.fc(X)
        return X

In [26]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

X = []
y = []
manipulated_path = './data/manipulated'
original_path = './data/original'
print("processing fake datasets")
for img_name in tqdm(os.listdir(manipulated_path)):
    X.append(np.array(Image.open(f'{manipulated_path}/{img_name}')))
    y.append([1, 0])
print("processing true datasets")
for img_name in tqdm(os.listdir(original_path)):
    X.append(np.array(Image.open(f'{original_path}/{img_name}')))
    y.append([0, 1])
X = np.array(X).swapaxes(3, 1).swapaxes(3, 2)
y = np.array(y)
print(X.shape)
print(y.shape)

processing fake datasets


100%|██████████| 8000/8000 [00:19<00:00, 415.04it/s]


processing true datasets


100%|██████████| 4000/4000 [00:09<00:00, 417.90it/s]


(12000, 3, 299, 299)
(12000, 2)


In [29]:
np.random.seed(42)
indices = np.random.permutation(X.shape[0])
X_shuffled = []
y_shuffled = []
for i in range(X.shape[0]):
    X_shuffled.append(X[indices[i]])
    y_shuffled.append(y[indices[i]])
X_shuffled = np.array(X_shuffled, dtype=np.float32)
y_shuffled = np.array(y_shuffled, dtype=np.float32)
train_X = X_shuffled[:int(X.shape[0] * 0.8)]
train_y = y_shuffled[:int(X.shape[0] * 0.8)]
valid_X = X_shuffled[int(X.shape[0] * 0.8):int(X.shape[0] * 0.9)]
valid_y = y_shuffled[int(X.shape[0] * 0.8):int(X.shape[0] * 0.9)]
test_X = X_shuffled[int(X.shape[0] * 0.9):]
test_y = y_shuffled[int(X.shape[0] * 0.9):]
print(train_X.shape[0])
print(valid_X.shape[0])
print(test_X.shape[0])

9600
1200
1200


In [102]:
import math

def process_epoch(X, y, model, opt=None, batch_size=100):
    if opt:
        model.train()
    else:
        model.eval()

    loss_func = nn.CrossEntropyLoss()
    correct, loss_sum, n_step, n_samples = 0, 0, 0, 0

    iter = math.ceil(X.shape[0] / batch_size)
    for batch in tqdm(range(iter)):
        X_batch = X[batch * batch_size:min((batch + 1) * batch_size, X.shape[0])][:, :, :224, :224]
        y_batch = y[batch * batch_size:min((batch + 1) * batch_size, X.shape[0])]
        if opt:
            opt.zero_grad()
        pred_y = model(torch.tensor(X_batch))
        loss = loss_func(pred_y, torch.tensor(y_batch))
        correct += np.sum(np.array(pred_y.detach().numpy().argmax(axis=1) == y_batch.argmax(axis=1), dtype=np.int64))
        if opt:
            loss.backward()
            opt.step()
        loss_sum += loss.detach().numpy()
        n_step += 1
        n_samples += X_batch.shape[0]

    return 1 - correct / n_samples, loss_sum / n_step

In [31]:
def train(
        batch_size=100,
        epochs=10,
        optimizer=torch.optim.Adam,
        lr=0.001,
        weight_decay=0.001,
        model=None):

    if(model == None):
        print("Please choose a type of ResNet!")
        return

    opt = optimizer(model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(epochs):
        print(f"Epoch {epoch}: ")
        train_acc, train_loss = process_epoch(train_X, train_y, model, batch_size=batch_size, opt=opt)
        test_acc, test_loss = process_epoch(test_X, test_y, model, batch_size=batch_size)
        print(f"Train Accuracy: {round(train_acc, 3)} Test Accuracy: {round(test_acc, 3)}")

In [105]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train(model=ResNet(ResidualModule, [3, 4, 6, 3]).to(device))

Epoch 0: 


  0%|          | 0/96 [00:00<?, ?it/s]

tensor([[0.4168, 0.5832],
        [0.4100, 0.5900],
        [0.4036, 0.5964],
        [0.3667, 0.6333],
        [0.3554, 0.6446],
        [0.4357, 0.5643],
        [0.3362, 0.6638],
        [0.3176, 0.6824],
        [0.4515, 0.5485],
        [0.3741, 0.6259]], grad_fn=<SliceBackward0>)


  1%|          | 1/96 [00:06<10:56,  6.92s/it]

tensor([[0.9925, 0.0075],
        [0.9982, 0.0018],
        [0.9978, 0.0022],
        [0.8908, 0.1092],
        [0.9965, 0.0035],
        [0.9968, 0.0032],
        [0.9912, 0.0088],
        [0.8783, 0.1217],
        [0.9949, 0.0051],
        [0.9976, 0.0024]], grad_fn=<SliceBackward0>)


...

tensor([[0.9760, 0.0240],
        [0.8964, 0.1036],
        [0.8561, 0.1439],
        [0.8971, 0.1029],
        [0.8908, 0.1092],
        [0.8847, 0.1153],
        [0.8757, 0.1243],
        [0.8888, 0.1112],
        [0.8881, 0.1119],
        [0.8868, 0.1132]], grad_fn=<SliceBackward0>)


 14%|█▎        | 13/96 [01:51<11:47,  8.52s/it]

tensor([[0.9132, 0.0868],
        [0.9155, 0.0845],
        [0.9893, 0.0107],
        [0.9238, 0.0762],
        [0.8972, 0.1028],
        [0.8870, 0.1130],
        [0.8976, 0.1024],
        [0.7755, 0.2245],
        [0.8778, 0.1222],
        [0.8996, 0.1004]], grad_fn=<SliceBackward0>)


 14%|█▎        | 13/96 [01:59<12:44,  9.21s/it]


KeyboardInterrupt: 